In [1]:
from buffer import * 
from database import * 
from pageNew import * 
from relation import * 

# Introduccion Actividad 2

En este control trabajaremos con una implementación de componentes básicos de una base de datos.

Se les entregará una implementación sencilla de un storage manager que maneja páginas de disco, y un buffer manager que actual como un intermediario entre los datos en el disco, y la base de datos. Adicionalmente, tendremos una libreria que maneja relaciones, y un motor sencillo de bases de datos que permite crear tablas, insertar tuplas, y hacer consultas sencillas.

**IMPORTANTE** : para ejecutar este jupyter notebook deben haber descargado los archivos disponibles en el github y tenerlos en la misma carpeta en la que se encuentra este notebook.   

Como vimos en la clase, el flujo de información es el siguiente: los datos se guardan en páginas de disco, y el buffer manager carga las páginas desde el disco para que la base de datos las pueda ocupar. 

En nuestro caso representaremos las paginas de disco por archivos que contienen 6 lineas donde: 
* las lineas 1 a 5 contienen las casillas
* la linea 6 contiene el string `"<NEXT> next_page"` que indica la siguiente página en el disco; "next_page" es o el nombre de un archivo en el disco, o `"<NULL>"` cuando estamos en la última página 
* cada casilla puede tener una tupla o estar vacia (esto ultimo se represanta con con el string `"<EMPTY>"`)
* las tuplas se representaran por strings
* los tipos de datos que puede tener una tupla son int o string que **no contenga espacios** (el espacio lo utilizaremos para separar los distintos datos de una tupla)

**Ejemplo**

Consideremos la relacion R(a string, b int) poblada con los siguientes datos

a | b
-- | --
hello | 1
bye | 2
hola | 5

Entonces la pagina en disco que contiene esta relacion tendra el siguiente formato.  


```
hello 1
bye 2
hola 5
<EMPTY>
<EMPTY>
<NEXT> <NULL>
```

En el caso anterior, toda la relacion R cabe en una sola pagina de disco. Veamos ahora el siguiente caso: 

a | b
-- | --
hello | 1
bye | 2
hola | 5
soy | 9
una | 10
relacion | 7
grande | 2

Como el tamaño de pagina soporta un maximo de 5 tuplas, entonces nuestra relacion quedara dividida en dos paginas de disco: `R-1` y `R-2`. 


Primero veamos `R-1`
```
hello 1
bye 2
hola 5
soy 9
una 10
<NEXT> R-2
```
y ahora `R-2`
```
relacion 7
grande 2 
<EMPTY>
<EMPTY>
<EMPTY>
<NEXT> <NULL>
```










## Clase Page

En primer lugar conoceremos la clase Page implementada en el archivo `pageNew.py`. Esta clase es la que se encarga de manejar las paginas en las cuales estaran guardados los datos (tuplas).

A continuacion se lista la signatura de la clase Page que incluye los metodos necesarios para leer las paginas a partir de los archivos, escribir en ellas 
 
```
Page
	Datos:
	- size (número de tuplas que caben en la página)
	- pname (nómbre del archivo en disco que contiene la página)
	- data = [] (las tuplas de la página, representadas como un string)
	- next = ('<NULL>' o nombre de la siguiente página)
	- dirty (Un booleano que nos dice si el contenido de la página cambió, y hay que guardar estos cambios en el disco -- ver discusion del buffer después)
	Metodos:
	- write_page() -- escribir la página al disco (si dirty == True)
	- read_from_disk() -- leer la página pname desde el disco (pname es un dato del objeto Page)
	- set_next_page(next_pname) -- setea la siguiente página, pone dirty = True
	- get_next_page_name() -- devuelve la siguiente página, o None si esta no existe
	- insert_into_pos(pos, tuple) -- si la posición pos está vacía, se inserta la tupla tuple en este posición, dirty = True
	- get_element_from_page(pos) -- devuelve la tupla en la posición pos
```


Veamos el uso basico de esta clase:


```
page = Page('P-1') # Inicializa una página vacía; este objeto todavía no está en el disco
page.read_from_disk() # Carga el contenida de la página P-1 desde el disco, si P-1 existe en el disco
page.write_page() # Si el bit dirty = True, escribe la página al disco
page.set_next_page(next_page) # Setea el valor del next = next_page
page.get_next_page_name() # Devuelve la página siguiente, o None si esta no existe
page.insert_into_pos(pos,tuple) # Inserta la tupla tuple (somo un string) a la posición pos, si esta está vacía
page.get_element_from_page(pos) # Devueleve la tupla en la posición pos si se trata de una posición válida (0-4)
```





Por ejemplo, para leer la pagina R-1 (que ya tenemos creada) hacemos: 

In [4]:
# Cargamos una pagina que ya existe

page = Page('R-1') #Crear la instancia de pagina
page.read_from_disk() #Cargar la pagina en disco, si no existe va a tirar un error

# Revisamos que contiene la pagina R-1
print(page.data)

['30 30', '16 22', '0 7', '1 3', '7 2']


## Buffer Manager

La segunda componente de un motor de bases de datos es un buffer (o un Buffer Manager para ser más preciso). Esta componente reserva una cantidad de RAM para cargar páginas desde el disco, y permitir a la base de datos trabajar con los datos. 

En terminos simples, un buffer es simplemente un arreglo que contiene páginas de nuestras relaciones. En este sentido, el buffer es un "scrap space" para trabajar con las páginas de nuestras relaciones, y es el único "snapshot" de las relaciones que la base de datos puede ver en un momento.

Cuando un proceso en la base de datos (por ejemplo, la consulta de un usuario) necesita cierta tupla, el motor tiene que encargarse de conocer la página en cual encontrar la tupla, y pedir a buffer esta página. 
* Si la página está en el buffer, el buffer manager devuelve un puntero a la página a la base de datos. 
* Si la página no está en el buffer, el buffer manager carga la página desde el disco, y de nuevo devuelve el puntero hacia la página a la base de datos.

Para recalcar: **la única manera que una base de datos puede acceder a los datos es a través del buffer**, y no leyendo los datos directamente desde el disco. Entonces, todos los procesos que tienen que ver con accesso a los datos, actualizaciones, etc. **se hacen necesariamente a través del buffer**.

En cierto sentido, las páginas en el buffer son la versión actual de los datos que el motor ve. Para esto, cada página tiene un bit llamado `dirty`, que nos dice si la página cambió, y cuando el buffer manager decide sacar una página desde el buffer, lo que hace es escribirla al disco si la página hubo cambios (aquí simplificamos un poco la lógica detras de manejo de páginas, pero en la parte del curso que habla sobre transacciones veremos esto en más detalles).

Nuestra implementacion del buffer se ubica en el archivo `buffer.py`, que define la clase `Buffer`. La clase `Buffer` encapsula el trabajo de un buffer manager, y contiene lo siguiente:


```
Buffer:
	Datos:
	- buffSize Tamaño de nuestro buffer (número de frames)
	- frames = [] Arreglo de tamaño buffSize que contiene las páginas
	- pages_dict = {} El diccionario que mapea nombres de páginas a frames (e.g. {'p1': 1}) para poder buscar si la página ya está en el buffer
	- occupied El conteo de frames ocupados por páginas
	- clock_pos La sigueinte posición a cual se argará la página; funciona como un reloj modular

	Metodos:
	- isFull() True si occupied == buffSize
	- create_page(pname) -- crea la página pname y la escribe al disco (solo si esta no existe)
	- fetch_page(pname) -- carga la página pname a un frame, o indica que misma ya está en el buffer
	- get_page(pname) -- devuelve el puntero a la página pname en el buffer (si necesario la carga desde el sico)
	- ** fetch_into_frame(pname,frame_num) -- carga la página pname en el frame frame_num
	- evict_page(pname) -- saca la página pname desde el buffer; si dirty = True la escribe al disco
	- flush_buffer() -- limpia el buffer y lo deja vacio
	- flush_and_fill(pname,num) -- limpia el buffer, y después carga num páginas partiendo desde la página pname; si se llega a una página sin la página siguiente antes de cargar num páginas, el proceso se detiene [este metodo solo es necesario para implementar el block nested loop join]
```

A continuacion veremos el uso basico de esta clase:


```
buffer = Buffer() # Crea un buffer nuevo con la capacidad de 4 páginas (limite bajo para hacer testing)
buffer.fetch_page(pName) # Carga la página pName desde el disco al buffer, si esta no está en el buffer
buffer.get_page(pName) # Retorna puntero a la página pName en el buffer (si la página no está en el buffer al llamar este metodo, se carga al buffer usando fetch_page())
buffer.create_page(pName) # Crea una página nueva llamada pName (si esta ya no existe en el disco), la carga al buffer, y devuelve el puntero a esta página
buffer.evict_page(pName) # saca la página pName desde el buffer; si es necesario la escribe al disco
buffer.flush_buffer() # Deja el buffer vacío. Si alguna de las paginas sufrio cambios y tiene su bit dirty, entonces esos cambios pasan a disco. 
```










Veamos un ejemplo de uso:

In [27]:
buffer = Buffer() # Creamos un buffer nuevo

print(buffer.frames)

[None, None, None, None]


Como podemos ver el buffer se encuentra vacio, luego podemos cargar una pagina. 

In [13]:
buffer.fetch_page("R-1")
print(buffer.frames)
print(buffer.pages_dict)

[<pageNew.Page object at 0x7fc83efd3160>, None, None, None]
{'R-1': 0}


Finalmente hacemos flush y dejamos el buffer vacio

In [14]:
buffer.flush_buffer()
print(buffer.frames)

[None, None, None, None]


Para crear una nueva pagina hacemos: 

In [22]:
new_page = buffer.create_page("N-1")
# Podemos ver que la pagina nueva esta vacia
print(new_page.data)

# Podemos crear otra pagina nueva y linkearla a la anterior
new_page2 = buffer.create_page("N-2")
new_page.set_next_page("N-2")
print(new_page.get_next_page_name())

buffer.flush_buffer() # vaciamos el buffer, con esto los cambios que efectuamos en las paginas se guardan en disco

['<EMPTY>', '<EMPTY>', '<EMPTY>', '<EMPTY>', '<EMPTY>']
N-2


In [28]:
# Revisemos que los cambios persistan

page_n1 = buffer.get_page("N-1") #le pedimos la pagina al buffer, si la pagina no esta, el buffer la va a buscar a disco

print(page_n1.get_next_page_name())

N-2


## Relaciones

La última libreria que necesitamos es un manager de tablas. El manager de tablas que usaremos se encuentra en el archivo `relation.py`, y representa la interfaz de una tabla en nuestra base de datos. Esta interfaz nos permitira navegar a traves de las relaciones, que se encuentran alojadas en paginas de disco (ver la clase `Page`). Para ir iterando sobre las tuplas de la relacion, la interfaz provee los metodos `open()`, `close()` y `next()` que nos permiten abrir, cerrar y avanzar dentro de la relacion. 


```
Relation:
	Datos:
	- buffer -- el buffer que la relación ocupará (nuestra base de datos ocupará un buffer para todas las relaciones)
	- rName -- nombre de la relación; un string 
	- attributes -- lista de nombres de atributos e.g. ['a','b'] 
	- types -- lista de tipos de atributos e.g. ['int','str'] -- solo soportamos int y str 
	- numAttrib = len(attributes) -- número de atributos
	- root_page = rName + '-1' -- nombre de la primera página de la relación (convención R-1, R-2, ... para la relacion R) 
	- current_page -- la página actual del cursor usado por .open(), .next() 
	- current_pos -- la posición en current_page del cursor usado por .open(), .next()

	Metodos:
	- open() -- como en las clases
	- has_next() -- true si hay una tupla partiendo desde la posición actuál
	- next() -- Devuelve la siguiente tupla si existe, si no None
	- close() -- como en las clases
	- get_attribute_types() -- devuelve dicionario con el esquema de la relación (e.g. para R(a int, b str) devuelve {'a': 'int', 'b': 'str'} )
	- get_individual_values(tup) -- si tup es string con una tupla de la relación devuelve el arreglo de valores; e.g. si tup = '1 Hola', R(a int, b str), entonces R.get_individual_values(tup) = [1,'Hola']
	- insert_tuple(tup) -- inserta la tupla tup en la relación (esto deben implementarlo ustedes)
```

Uso básico de la clase Relation


```
R = Relation('R',['a','b'],['int','int'],buff) # Crea una nueva relación llamada 'R', con esquema R(a int, b int), ocupando el buffer buff para comunicarse con el disco; si la primera página de R (llamada 'R-1' existe en el disco, se asume que la relación ya existe, y se carga desde el disco, si no, se crea la página 'R-1', y escribe al disco)
R.open() # Se posiciona antes de la primera tupla de R; crea nuestro cursor
R.has_next() # Devuelve True si existe una siguiente tupla en R (desde la posición actual del cursor)
R.next() # Devuelve la siguiente tupla y mueve el cursor
R.close() # cierra el cursor para R
R.get_attribute_types() # Devuelve el diccionario indicando el tipo de cada atributo... {'a':'int', 'b':'int'} para nuestro ejemplo
R.get_individual_values(tup) # Si 'tup' es una tupla segun el esquema de 'R', se devuelve el diccionario de valores; por ejemplo, R.get_individual_values('1 73') devuelve {'a':1, 'b':73} en nuestro ejemplo
```

Veamos un ejemplo: 


In [ ]:
R = Relation('R' ,['a','b'], ['int','int'], buffer) # Cargamos la relacion R de disco
R.open()

In [ ]:
## Podemos revisar que la pagina quedo cargada en el buffer
print(buffer.frames)

[<pageNew.Page object at 0x7f4a2a2b3be0>, None, None, None]


In [ ]:
# Ahora imprimimos por las tuplas de R, esto nos mueve por todas las paginas de R (R-1, R-2, ... , R-n)
R.open()

while (R.has_next()):
  print(R.next())

R.close()

30 30
16 22
0 7
1 3
7 2
1 1
7 7
1 2
1 8
16 37
15 7
15 4
15 3
15 2
15 1
15 5
16 3
16 2
16 1
15 6
11 22


## Motor de base de datos

Finalmente, nuestro motor de bases de datos, ubicado en database.py, nos permite interactuar con las relaciones, cuyos datos están guardados en la páginas del disco, y cuales se consiguen ocupando el buffer.

La base de datos guarda dos objetos:
1. Buffer: un objeto de la clase Buffer que contiene nuestro buffer
2. Relations: un dicionario objetos de la clase Relation, indexado por el nombre de la relación (e.g. {'R': Relation('R',['a','b'],['int','int'])})

La base de datos provee las siguientes funcionalidades:

	- create_table(rName, attributes, types, buff = Buffer) -- se crea un objeto de tipo Relation que ocupa el buffer de la base de datos, y se guarda en el dicionario Relations. Si la relación con el nombre rName ya existe en la base de datos se devuelve un error
	- all_tuples(R) -- devuelve todas las tuplas de la relación R
	- filtered_tuples(R,cond) -- [ustedes deben implementarla] resuelve la consulta 'SELECT * FROM R WHERE a = x' aquí R es el nombre de la relación, y 'cond' es un condición del tipo 'attributo = valor'
	- cross(R, S) -- devuelve el producto cruz entre R y S; bajo la suposición que R no es igual a S
	- nested_loop_join(R,S,attrib) -- [ustedes deben implementarla] 'SELECT * FROM R,S WHERE R.attrib = S.attrib', supone que R no es igual a S

Veamos un ejemplo de uso: 


In [29]:
# Creando tablas (los archivos de estas dos relaciónes están en la carpeta con el código):
create_table('R',['a','b'],['int','int'])
create_table('S',['b','c'],['int','int'])

True

In [30]:
cross('R','S') # Imprime las tuplas en el producto cruz de nuestras dos relaciones

30 30 1 1
30 30 1 3
30 30 4 4
30 30 1 8
16 22 1 1
16 22 1 3
16 22 4 4
16 22 1 8
0 7 1 1
0 7 1 3
0 7 4 4
0 7 1 8
1 3 1 1
1 3 1 3
1 3 4 4
1 3 1 8
7 2 1 1
7 2 1 3
7 2 4 4
7 2 1 8
1 1 1 1
1 1 1 3
1 1 4 4
1 1 1 8
7 7 1 1
7 7 1 3
7 7 4 4
7 7 1 8
1 2 1 1
1 2 1 3
1 2 4 4
1 2 1 8
1 8 1 1
1 8 1 3
1 8 4 4
1 8 1 8
16 37 1 1
16 37 1 3
16 37 4 4
16 37 1 8
15 7 1 1
15 7 1 3
15 7 4 4
15 7 1 8
15 4 1 1
15 4 1 3
15 4 4 4
15 4 1 8
15 3 1 1
15 3 1 3
15 3 4 4
15 3 1 8
15 2 1 1
15 2 1 3
15 2 4 4
15 2 1 8
15 1 1 1
15 1 1 3
15 1 4 4
15 1 1 8
15 5 1 1
15 5 1 3
15 5 4 4
15 5 1 8
16 3 1 1
16 3 1 3
16 3 4 4
16 3 1 8
16 2 1 1
16 2 1 3
16 2 4 4
16 2 1 8
16 1 1 1
16 1 1 3
16 1 4 4
16 1 1 8
15 6 1 1
15 6 1 3
15 6 4 4
15 6 1 8
11 22 1 1
11 22 1 3
11 22 4 4
11 22 1 8


True

In [31]:
all_tuples('R') # Imprime todas las tuplas en 'R'

30 30
16 22
0 7
1 3
7 2
1 1
7 7
1 2
1 8
16 37
15 7
15 4
15 3
15 2
15 1
15 5
16 3
16 2
16 1
15 6
11 22


In [32]:
# Podemos crear una nueva relacion N, con las paginas que creamos anteriormente
# Nuestra relacion sera N(a int, b int, c int)

create_table('N',['a','b','c'],['int','int','int'])

True

In [34]:
# Como nuestras paginas estaban vacias, nuestra relacion tambien lo estara
all_tuples('N')

# El metodo para insertar tuplas deben implementarlo ustedes!! 

# La actividad

Para esta actividad ustedes deben implementar los siguientes metodos: 




1. **[2 puntos]** Implementar insert_tuple(tup) en `relation.py` 

Para esta primera actividad deben en primer lugar revisar si la tupla ya esta en la relacion. Si no está deben insertar la tupla en alguna posición que se encuentre vacia (sea una posicion de una pagina existente marcada con `'<EMPTY>'` o una posicion en una pagina nueva). Si crean una página nueva tienen que linkearla con la anterior. 

La manipulacion de paginas deben realizarla usando el buffer. Los metodos que usaran son: 
```
	- buffer.get_page(pname) -- para conseguir las páginas
	- page.insert_into_pos(emptyPos,tup)  -- para insertar un dato en una posición vacia
	- buffer.evict_page(pname) -- para forzar al buffer escribir una página al disco (una vez insertada la tupla nueva)
	- buffer.create_page(pname) -- si necesitan crear una página nueva
	- page.set_next_page(pname) -- para setear next_page
```
2. **[2 puntos]** Implementar el método filtered_tuples(R,cond) en `database.py`
3. **[2 puntos]** Implementar el método nested_loop_join(R,S) en `database.py`

Para estas dos actividades pueden utilizar los metodos `open(), next(), has_next(), get_attribute_types(), get_individual_value()` de la clase `Relation` (ademas de  metodos de string)





 

## La entrega

Deben entregar un archivo .zip que contenga los archivos .py con las implementaciones pedidas. Ademas debe entregar este jupyter notebook con un demo de codigo que realice lo siguiente: 

1. Crear una relacion nueva  T(a int, c int)
2. Usando su metodo `insert_tuple(tup)`, inserte 7 tuplas en T. Luego de esto, muestre todas las tuplas en T
3. Usando su metodo `filtered_tuples(R,cond)` realice la consulta `SELECT * FROM T WHERE a=1`
4. Ejecute un nested loop join entre R y T
5. Ejecute un nested loop join entre S y T

Si lo considera necesario, escriba un readme que explique su codigo y/o como ejecutarlo. 

**Recuerde que los datos de las relaciones R y S ya estan creados y disponibles para su descarga en el github del curso (no deben crearlos ustedes)**



*Escriba aqui su readme!*
> *bla, bla bla*



In [ ]:
## Escriba aqui el demo de uso del metodo insert_tuple(tup)

# Crear la relacion R
# Hacer 7 inserciones
# Mostrar all_tuples('T')


In [ ]:
## Escriba aqui el demo de uso del metodo filtered_tuples(R,cond)

# SELECT * FROM T WHERE a=1

In [ ]:
## Escriba aqui el demo de uso del metodo nested_loop_join(R,S)

# nested_loop_join(R,T)

# nested_loop_join(S,T)

# Bonus

**Bonus 1 [0.5 puntos]**

Para todas las operaciones de join estamos suponemos que las relaciones son distintas. Esto ocurre por nuestra interfaz de la clase Relation, que maneja un objeto y un iterador a la vez.

El primer bonus es implementar un metodo `get_iterator()` en la clase Relation, que nos devuelve un objeto con metodos `open()`, `next()`, `has_next()` y `close()`, y nos permite iterar independiente sobre las tuplas de la relación. En particular, si en nuestra implementación de `cross(R,S)` reemplazamos 

```
    R = Relations[R]
    S = Relations[S]
```
por

```
    R = Relations[R].get_iterator()
    S = Relations[S].get_iterator()
```



**Bonus 2 [1.5 puntos]**

El segundo bonus es implementar el algoritmo de `block_nested_loop_join(R,S)` usando el buffer. Para esto pueden ocupar todos los metodos y datos de la clase Buffer

In [ ]:
## Escriba aqui un demo de uso del metodo get_iterator()

In [ ]:
## Escriba aqui un demo de uso del metodo block_nested_loop_join()